<a href="https://www.kaggle.com/code/rraydata/use-less-gpu-resource-to-fine-tune-llama-and-llama?scriptVersionId=147886602" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Use less GPU resource to Fine Tune LLAMA and LLAMA2 

Today we'll explore fine-tuning the Llama 2 model available on Kaggle Models using Multi-lora.

-  LoRa: [Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685): freezes the pre-trained model weights and injects trainable rank decomposition matrices into each layer of the Transformer architecture, greatly reducing the number of trainable parameters for downstream tasks.
- QloRA: [Quantized Low Rank Adapters](https://arxiv.org/abs/2305.14314):QLoRA backpropagates gradients through a frozen, 4-bit quantized pretrained language model into Low Rank Adapters~(LoRA).While the base model is quantized with NF4, the trained LoRA’s parameters remain at a higher precision which is usually FP16. 


Disadvantages of LoRA-based Approaches:

- Memory Consumption: Some Lora techniques might be efficient, but the introduction of low-rank approximations can sometimes heighten memory usage, particularly if one has to store both original and approximated parameters.
- Potential for Reduced Model Accuracy: Lora-based fine-tuning is designed to either maintain or boost model accuracy. However, there can be instances where the approximations cause a dip in performance, especially if the low-rank approximations aren't chosen carefully.
- Dependence on Hyperparameters: Much like other ML techniques, Lora-based strategies involve hyperparameters that need precise fine-tuning. Mistakes in this area can lead to subpar performance.


[ASPEN: Efficient LLM Model Fine-tune and Inference via Multi-Lora Optimization](https://github.com/TUDB-Labs/multi-lora-fine-tune#experiment-results) is an open-source framework for fine-tuning Large Language Models (LLMs) using the efficient multiple LoRA/QLoRA methods. How Multi-lora improve upon LoRA-based approaches:

- GPU Memory Conservation: Use one foundational model for multiple fine tuning process, significantly saving resources.
- Automatic Parameter Learning: Introducing automation in the learning process for hyperparameters during model fine-tuning can speed up the process and guarantee optimal model results.
- Early Stopping Mechanism: Implementing this approach ensures no overfitting occurs, and resources are utilized effectively. It stops training once the model's improvement becomes negligible.

## 1. Clone multi-lora repository

In [1]:
import os
os.chdir('/kaggle/working/')

In [2]:
!git clone https://github.com/TUDB-Labs/multi-lora-fine-tune.git

Cloning into 'multi-lora-fine-tune'...
remote: Enumerating objects: 621, done.
remote: Counting objects: 100% (308/308), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 621 (delta 202), reused 253 (delta 189), pack-reused 313
Receiving objects: 100% (621/621), 2.21 MiB | 11.17 MiB/s, done.
Resolving deltas: 100% (332/332), done.


## 2. Install dependencies

In [3]:
!pip install -r /kaggle/working/multi-lora-fine-tune/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 3. Config finetune datasets and parameters. You can add multiple lora parameters and datasets.

ASPEN can be used on:

1) Domain-Specific Fine-Tuning:  This involves adapting a single model with various parameters particularly for one domain.

2) Cross-Domain Fine-Tuning: This approach utilizes the foundational model to optimize multiple models, each designed for diverse domains, by incorporating datasets from various or identical domains.

The demo data and prompt are for demonstration purpose.

In [4]:
!cat /kaggle/working/multi-lora-fine-tune/data/data_demo.json

[
    {
        "instruction": "Instruction demo.",
        "input": "Input demo.",
        "output": "Output demo."
    },
    {
        "instruction": "Instruction demo.",
        "output": "Output demo."
    }
]

In [5]:
!cat /kaggle/working/multi-lora-fine-tune/template/template_demo.json

{
    "description": "",
    "parameter": [
        "input",
        "output",
        "instruction"
    ],
    "prompt": "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Output:\n{output}\n",
    "prompt_no_input": "### Instruction:\n{instruction}\n\n### Output:\n{output}\n"
}

In [6]:
config_string = """
{
    "cutoff_len": 256,
    "group_by_length": false,
    "expand_right": true,
    "pad_token_id": -1,
    "save_step": 2000,
    "early_stop_test_step": 2000,
    "train_lora_candidate_num": 4,
    "train_lora_simultaneously_num": 2,
    "train_strategy": "optim",
    "lora": [
        {
            "name": "lora_0",
            "output": "lora_0",
            "optim": "adamw",
            "lr": 3e-4,
            "batch_size": 16,
            "micro_batch_size": 4,
            "test_batch_size": 64,
            "num_epochs": 3,
            "r": 8,
            "alpha": 16,
            "dropout": 0.05,
            "target_modules": {
                "q_proj": true,
                "k_proj": false,
                "v_proj": true,
                "o_proj": false,
                "w1_proj": false,
                "w2_proj": false,
                "w3_proj": false
            },
            "data": "/kaggle/working/multi-lora-fine-tune/data/data_demo.json",
            "test_data": "/kaggle/working/multi-lora-fine-tune/data/data_demo.json",
            "prompt": "/kaggle/working/multi-lora-fine-tune/template/template_demo.json"
        },
        {
            "name": "lora_1",
            "output": "lora_1",
            "optim": "adamw",
            "lr": 3e-4,
            "batch_size": 16,
            "micro_batch_size": 4,
            "test_batch_size": 64,
            "num_epochs": 3,
            "r": 8,
            "alpha": 16,
            "dropout": 0.05,
            "target_modules": {
                "q_proj": true,
                "k_proj": false,
                "v_proj": true,
                "o_proj": false,
                "w1_proj": false,
                "w2_proj": false,
                "w3_proj": false
            },
            "data": "/kaggle/working/multi-lora-fine-tune/data/data_demo.json",
            "test_data": "/kaggle/working/multi-lora-fine-tune/data/data_demo.json",
            "prompt": "/kaggle/working/multi-lora-fine-tune/template/template_demo.json"
        }
    ]
}
"""

with open("./config.json", "w") as f:
    f.write(config_string)

 ## 4. Add the path of the base model and config file path to start finetune

remember to check whether the section is on GPU. 

In [7]:
!python /kaggle/working/multi-lora-fine-tune/mlora.py \
  --base_model /kaggle/input/llama-2/pytorch/7b-hf/1 \
  --config /kaggle/working/config.json \
  --load_8bit


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
ASPEN requires NVIDIA CUDA computing capacity. Please check your PyTorch installation.


## 5. Then two files(lora_0, lora_1) appear in the current directory, that's the finetuned model adapter. We can download them.